In [117]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan

In [118]:
spark

## Read data

### Load files from google cloud storage

In [119]:
sdf_assets = spark.read.json("gs://lz-assets/*.json")

In [149]:
sdf_orders = spark.read.json("gs://lzs-order/*.json")

### Save data to csv for local debugging

In [9]:
#sdf_assets.filter(sdf_assets.CustomerId.isNotNull()).limit(100).toPandas().to_csv("assets.csv")

In [11]:
#sdf_orders.filter(sdf_orders.CustomerId.isNotNull()).limit(100).toPandas().to_csv("orders.csv")

## Warranty data

### Pivot warranty data - group by customer and have each product as own column

In [5]:
sdf_assets.filter(sdf_assets.CustomerId.isNotNull()).limit(3).toPandas()

,CustomerId,Id,ManufactureDate,MyFestoolId,ProductCode,ProductName,PurchaseDate,RegistrationDate,SalesNumber,Source,Status
0,879cb75b-2568-41aa-b05b-5d3e12d85cf8,02i3X00000iy01eQAA,2019-03-14T00:00:00,f08446b1-3808-482b-9a9e-0ec9917e6015,None,TS 55 REBQ 230V,2019-08-23T00:00:00,2019-08-23T00:00:00+00:00,00561551,ERP,Active
1,879cb75b-2568-41aa-b05b-5d3e12d85cf8,02i3X00000iyjkQQAQ,2019-04-08T00:00:00,f08446b1-3808-482b-9a9e-0ec9917e6015,None,CMS-GE 230V,2019-04-29T00:00:00,2019-04-30T00:00:00+00:00,00561228,Internet,Active
2,879cb75b-2568-41aa-b05b-5d3e12d85cf8,02i3X00000iygZGQAY,2018-03-06T00:00:00,f08446b1-3808-482b-9a9e-0ec9917e6015,None,MFK 700 EQ 230V-240V,2018-09-23T00:00:00,2020-07-15T00:00:00+00:00,00574369,Manually,Active


In [7]:
sdf_assets_items = sdf_assets.filter(sdf_assets.CustomerId != 'null').groupBy("CustomerId").agg(F.collect_set("SalesNumber").alias("SalesNumbers"))

## Filter for top10 products / top10 machines (no spare parts)

In [161]:
sdf_assets.groupBy("SalesNumber").count().orderBy(F.col("count").desc()).toPandas().head(11)

,SalesNumber,count
0,None,126
1,00561445,15
2,00571805,15
3,00561160,13
4,00561551,13
5,00564531,10
6,00575279,10
7,00574369,9
8,00583490,9
9,00574832,9


In [155]:
sdf_assets_items.limit(10).toPandas()

,CustomerId,SalesNumbers
0,f9497eaa-7d2d-4fd9-bbbb-3b545d69e31b,"[00577649, 00577426, 00576895, 00577600]"
1,39122c18-7fa8-48a3-92d5-5cc97e8a8c6f,"[00575990, 00574984]"
2,a7eb0ce4-7f34-4a25-bf60-ac218acc1687,"[00574988, 00576029, 00576207, 00574776]"
3,a2da1169-c66f-466e-9b22-53f29b705e1b,"[00574341, 00574635, 00574709]"
4,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,"[00576041, 00574832, 00576072, 00577241, 00574..."
5,50534e9e-d2fe-4596-a893-1e1b4b020f83,"[00576825, 00577430, 00577415, 00576703]"
6,9fa78774-38fb-4a71-9a6d-f23ca4928806,"[00561396, 00561579, 00575254, 00561180, 00561..."
7,fd8ee07e-b5de-4184-bee8-dfc63e2ecaf1,"[00574341, 00574822, 00574723, 00577050, 00574..."
8,1878ab58-35c1-497e-a409-6db9fe819d52,"[00571934, 00574983, 00576370, 00574984, 00575..."
9,310442c2-80e3-471f-be9e-cc8ad352d050,"[00576041, 00574765, 00576615, 00564271, 00571..."


In [162]:
values_to_check = [ '00561445', '00571805', '00561160', '00561551', '00564531', '00575279', '00574369', '00583490','00574832', '00574341' ]
     
# Create a filter condition for each value in the list
filter_condition = None
for value in values_to_check:
    if filter_condition is None:
        filter_condition = F.array_contains(sdf_assets_items.SalesNumbers, value)
    else:
        filter_condition = filter_condition | F.array_contains(sdf_assets_items.SalesNumbers, value)

# Filter rows where "SalesNumbers" contains any of the specified values
filtered_df = sdf_assets_items.filter(filter_condition)

In [163]:
filtered_df.limit(10).toPandas()

,CustomerId,SalesNumbers
0,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,"[00576041, 00574832, 00576072, 00577241, 00574..."
1,9fa78774-38fb-4a71-9a6d-f23ca4928806,"[00561396, 00561579, 00575254, 00561180, 00561..."
2,fd8ee07e-b5de-4184-bee8-dfc63e2ecaf1,"[00574341, 00574822, 00574723, 00577050, 00574..."
3,a2da1169-c66f-466e-9b22-53f29b705e1b,"[00574341, 00574635, 00574709]"
4,9f3a9794-af8c-4fb8-b2b1-0fbb27910897,"[00575279, 00575781, 00576329, 00576820, 00575..."
5,879cb75b-2568-41aa-b05b-5d3e12d85cf8,"[00561551, 00570203, 00576684, 00576829, 00574..."
6,310442c2-80e3-471f-be9e-cc8ad352d050,"[00576041, 00574765, 00576615, 00564271, 00571..."
7,e76240a2-71e6-4cfb-ba2b-3a0d5af97a7b,"[00574335, 00576678, 00574605, 00571870, 00583..."
8,d2bcbc0a-d14a-4995-9712-01f6e17f1c07,"[00564532, 00577033, 00561461, 00574832, 00576..."
9,df445fa7-195f-4d67-8f75-32b6b544860b,"[00575781, 00574832, 00577053, 00576601, 00575..."


## Use FP Growth to create association rules

In [181]:
fpGrowth = FPGrowth(itemsCol="SalesNumbers", minSupport=0.12, minConfidence=0.1)
model = fpGrowth.fit(filtered_df)

# Display frequent itemsets.
model.freqItemsets.sort(F.desc("items")).toPandas()

,items,freq
0,[00583490],3
1,"[00575781, 00575302]",3
2,[00575781],3
3,[00575302],5
4,[00574832],4
5,[00574453],3
6,[00574369],3
7,[00574357],3
8,[00574341],4
9,[00574335],3


In [188]:
# Display generated association rules.
model.associationRules.sort("antecedent", "consequent").toPandas()

,antecedent,consequent,confidence,lift,support
0,[00561136],[00571805],1.00,4.25,0.176471
1,[00571805],[00561136],0.75,4.25,0.176471
2,[00575302],[00575781],0.60,3.40,0.176471
3,[00575781],[00575302],1.00,3.40,0.176471


Lift may have more digits than shown here

In [193]:
model.associationRules.dropDuplicates(subset=["lift", "support"]).toPandas()

,antecedent,consequent,confidence,lift,support
0,[00575302],[00575781],0.6,3.40,0.176471
1,[00575781],[00575302],1.0,3.40,0.176471
2,[00561136],[00571805],1.0,4.25,0.176471


In [167]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(filtered_df).toPandas()

,CustomerId,SalesNumbers,prediction
0,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,"[00576041, 00574832, 00576072, 00577241, 00574...","[00576247, 00576601]"
1,9fa78774-38fb-4a71-9a6d-f23ca4928806,"[00561396, 00561579, 00575254, 00561180, 00561...","[00574723, 00574341, 00561283]"
2,fd8ee07e-b5de-4184-bee8-dfc63e2ecaf1,"[00574341, 00574822, 00574723, 00577050, 00574...","[00561180, 00564532, 00561228, 00575302, 00768..."
3,310442c2-80e3-471f-be9e-cc8ad352d050,"[00576041, 00574765, 00576615, 00564271, 00571...","[00561228, 00561551, 00561136]"
4,879cb75b-2568-41aa-b05b-5d3e12d85cf8,"[00561551, 00570203, 00576684, 00576829, 00574...","[00575781, 00576820, 00561180, 00574557, 00564..."
5,a2da1169-c66f-466e-9b22-53f29b705e1b,"[00574341, 00574635, 00574709]","[00574723, 00561228, 00564531, 00575302, 00768..."
6,9f3a9794-af8c-4fb8-b2b1-0fbb27910897,"[00575279, 00575781, 00576329, 00576820, 00575...","[00561228, 00574341, 00768809]"
7,d2bcbc0a-d14a-4995-9712-01f6e17f1c07,"[00564532, 00577033, 00561461, 00574832, 00576...","[00576601, 00561180, 00564531]"
8,e83284f4-1d02-42b0-8175-8c1de2c53bb4,"[00574364, 00564164, 00574180, 00574228, 00574...",[00571870]
9,e76240a2-71e6-4cfb-ba2b-3a0d5af97a7b,"[00574335, 00576678, 00574605, 00571870, 00583...","[00574357, 00561228, 00574341, 00575781, 00768..."


In [196]:
model.save('gs://lz-gcs/fp_growth_assets')

----------------------------------------                                        
Exception happened during processing of request from ('127.0.0.1', 37002)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 239, in accum_updat

## Pivot dataframe

In [67]:
sdf_assets_pivot = sdf_assets.filter(sdf_assets.CustomerId != 'null').groupBy("CustomerId").pivot("SalesNumber").agg(F.count("SalesNumber"))

Cleanup null values in columns and rows.

In [68]:
columns_to_drop = [col_name for col_name in sdf_assets_pivot.columns if col_name == 'null']

# Drop columns with null names or null values
sdf_assets_pivot = sdf_assets_pivot.drop(*columns_to_drop)

Strip leading zeros to align with order data

In [69]:
columns_to_modify = [col_name for col_name in sdf_assets_pivot.columns if col_name != 'CustomerId']
for col_name in columns_to_modify:
    sdf_assets_pivot = sdf_assets_pivot.withColumnRenamed(col_name, col_name.lstrip('0'))

The other dataframe produces NaN values, this one None, so align them.

In [70]:
sdf_assets_pivot = sdf_assets_pivot.select([F.when(F.col(c).isNull(), 0).otherwise(F.col(c)).alias(c) for c in sdf_assets_pivot.columns])

In [71]:
sdf_assets_pivot.limit(10).toPandas()

23/10/07 08:28:51 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,CustomerId,205034,495315,561100,561136,561138,561162,561180,561228,561261,...,764986,767991,767996,767997,768008,768809,768997,769531,769532,769533
0,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,879cb75b-2568-41aa-b05b-5d3e12d85cf8,0,0,0,0,0,0,0,4,1,...,0,1,0,1,0,1,0,2,1,2
2,9fa78774-38fb-4a71-9a6d-f23ca4928806,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fd8ee07e-b5de-4184-bee8-dfc63e2ecaf1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,39122c18-7fa8-48a3-92d5-5cc97e8a8c6f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,a7eb0ce4-7f34-4a25-bf60-ac218acc1687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1878ab58-35c1-497e-a409-6db9fe819d52,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,310442c2-80e3-471f-be9e-cc8ad352d050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,f9497eaa-7d2d-4fd9-bbbb-3b545d69e31b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,50534e9e-d2fe-4596-a893-1e1b4b020f83,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Test with ALS recommender

In [102]:
from pyspark.sql import Row

data = [
    Row(userId=0, itemId=575990, rating=1),
    Row(userId=0, itemId=574984, rating=1),
    Row(userId=0, itemId=577426, rating=0),
    
    Row(userId=1, itemId=577649, rating=1),
    Row(userId=1, itemId=575990, rating=1),
    Row(userId=1, itemId=577426, rating=1),
    
    Row(userId=2, itemId=577649, rating=0),
    Row(userId=2, itemId=575990, rating=0),
    Row(userId=2, itemId=577426, rating=1),
]

df = spark.createDataFrame(data)

training, test = df.randomSplit([0.8, 0.2])

In [112]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="itemId", ratingCol="rating")

model = als.fit(training)

predictions = model.transform(test)

In [113]:
user_id = 2
recommendations = model.recommendForUserSubset(spark.createDataFrame([(user_id,)], ["userId"]), numItems=5)
recommendations.toPandas()

,userId,recommendations
0,2,"[(574984, 9.518616025161464e-06), (575990, 4.7..."


In [115]:
model.recommendForAllUsers(10).toPandas()

,userId,recommendations
0,1,"[(574984, 2.4570479581598192e-05), (575990, 1...."
1,2,"[(574984, 9.518616025161464e-06), (575990, 4.7..."
2,0,"[(574984, 0.0004948157002218068), (575990, 0.0..."


In [116]:
model.recommendForAllItems(10).toPandas()

,itemId,recommendations
0,575990,"[(0, 0.000247489515459165), (1, 1.228927703778..."
1,577649,"[(0, 1.5433162843692116e-05), (1, 6.7006162680..."
2,574984,"[(0, 0.0004948157002218068), (1, 2.45704795815..."
3,577426,"[(0, 2.1411697161966003e-05), (1, 9.2966020019..."


## Orders

In [150]:
sdf_orders.filter(sdf_orders.CustomerId.isNotNull()).limit(3).toPandas()

,AnonymousId,CartId,CountryKey,CreatedAt,CustomerId,Id,LastModifiedAt,LineItems,Locale,OrderNumber,...,ShippingPrice,ShippingPriceTotalGross,ShippingState,State,Store,TaxMode,TotalGross,TotalNet,TotalPrice,TotalTax
0,ea85f5a4-44c5-490b-b047-09d3a9f7c249,5974ae3f-2c86-4861-921e-6ffdd19e0bb8,DE,2023-10-02T04:29:43.038Z,04c72e7c-aabb-4055-ac51-0af12068cca5,8c77b674-aa35-42e4-9ec5-4a6d82f851cf,2023-10-02T23:02:56.063Z,"[(2023-09-26T17:08:14.277Z, (6, None, True, 1)...",de,44441485,...,0.00,0.0,Shipped,Complete,storeDE,Platform,301.73,253.55,253.55,48.18
1,None,805f9179-de59-4bd2-aae4-61420416e8cb,DE,2023-09-28T23:26:26.453Z,e48a969b-265f-4e7c-b75d-ffeb4aeb1fd0,75b7db27-f850-418b-9f1e-d9d45015524e,2023-09-29T23:02:54.231Z,"[(2023-09-28T22:41:36.088Z, (999, None, True, ...",de,87941535,...,4.96,5.9,Shipped,Complete,storeDE,Platform,40.41,33.96,33.96,6.45
2,1fe8e8ce-927b-424f-9def-c876db3c147f,c15e8f25-25a7-4161-81d6-74cb93ea1b52,DE,2023-10-03T07:48:35.093Z,ed7653e4-67ee-414f-9884-354a406b9ea1,a6b0b5aa-ee96-4d83-bf0b-31fae77efa73,2023-10-04T23:03:15.172Z,"[(2023-10-03T07:46:56.006Z, (917, None, True, ...",de,92540147,...,0.00,0.0,Shipped,Complete,storeDE,Platform,54.68,45.95,45.95,8.73


In [151]:
sdf_orders.count()

437

Select line items, we want a data frame where each row is one line item

### No need for a JSON schema because the data is already parsed

In [56]:
sdf_orders.select("LineItems")

DataFrame[LineItems: array<struct<AddedAt:string,Availability:struct<AvailableQuantity:bigint,Channels:string,IsOnStock:boolean,RestockableInDays:bigint>,Id:string,ImageUrl:string,LastModifiedAt:string,Name:string,Price:struct<Discounted:string,Value:double>,ProductId:string,ProductNumber:string,Tax:double,TaxedGrossPrice:double,TaxedNetPrice:double,TypeId:string>>]

Explode the array of line items to create multiple rows

In [30]:
sdf_orders_exploded = sdf_orders.select(F.col("CustomerId"), F.col("OrderNumber"), F.explode(F.col("LineItems")).alias("LineItem"))

In [31]:
sdf_orders_exploded

DataFrame[CustomerId: string, OrderNumber: string, LineItem: struct<AddedAt:string,Availability:struct<AvailableQuantity:bigint,Channels:string,IsOnStock:boolean,RestockableInDays:bigint>,Id:string,ImageUrl:string,LastModifiedAt:string,Name:string,Price:struct<Discounted:string,Value:double>,ProductId:string,ProductNumber:string,Tax:double,TaxedGrossPrice:double,TaxedNetPrice:double,TypeId:string>]

In [32]:
sdf_orders_exploded.limit(5).toPandas()

,CustomerId,OrderNumber,LineItem
0,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,"(2023-09-26T17:08:14.277Z, (6, None, True, 1),..."
1,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,"(2023-09-26T17:08:51.328Z, (16, None, True, 1)..."
2,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,"(2023-09-26T17:09:02.684Z, (1000, None, True, ..."
3,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,"(2023-09-26T17:09:17.453Z, (1000, None, True, ..."
4,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,"(2023-09-26T17:09:25.416Z, (13, None, True, 1)..."


Access the individual fields within the struct

In [33]:
sdf_orders_exploded = sdf_orders_exploded.select("CustomerId", "OrderNumber", "LineItem.TypeId", "LineItem.ProductNumber", "LineItem.Name", "LineItem.Price.Value", "LineItem.Price.Discounted", "LineItem.AddedAt", "LineItem.Availability.IsOnStock")

In [34]:
sdf_orders_exploded

DataFrame[CustomerId: string, OrderNumber: string, TypeId: string, ProductNumber: string, Name: string, Value: double, Discounted: string, AddedAt: string, IsOnStock: boolean]

In [35]:
sdf_orders_exploded.limit(5).toPandas()

,CustomerId,OrderNumber,TypeId,ProductNumber,Name,Value,Discounted,AddedAt,IsOnStock
0,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,d490fc5d-7fe1-4175-b08a-3b38443b9116,10026390,None,5.9,None,2023-09-26T17:08:14.277Z,True
1,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,d490fc5d-7fe1-4175-b08a-3b38443b9116,10711428,None,95.2,None,2023-09-26T17:08:51.328Z,True
2,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,d490fc5d-7fe1-4175-b08a-3b38443b9116,482098,None,1.5,None,2023-09-26T17:09:02.684Z,True
3,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,d490fc5d-7fe1-4175-b08a-3b38443b9116,228546,None,1.0,None,2023-09-26T17:09:17.453Z,True
4,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,d490fc5d-7fe1-4175-b08a-3b38443b9116,451630,None,3.1,None,2023-09-26T17:09:25.416Z,True


Replace TypeIDs:
- `03f9ec1f-502b-44e3-a9fa-7d2078064198` => Product
- `d490fc5d-7fe1-4175-b08a-3b38443b9116` => SparePart

In [37]:
sdf_orders_exploded = sdf_orders_exploded.withColumn("TypeId", F.regexp_replace(F.col("TypeId"), "d490fc5d-7fe1-4175-b08a-3b38443b9116", "SparePart"))
sdf_orders_exploded = sdf_orders_exploded.withColumn("TypeId", F.regexp_replace(F.col("TypeId"), "03f9ec1f-502b-44e3-a9fa-7d2078064198", "Product"))

In [61]:
sdf_orders_exploded.limit(5).toPandas()

,CustomerId,OrderNumber,TypeId,ProductNumber,Name,Value,Discounted,AddedAt,IsOnStock
0,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,SparePart,10026390,None,5.9,None,2023-09-26T17:08:14.277Z,True
1,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,SparePart,10711428,None,95.2,None,2023-09-26T17:08:51.328Z,True
2,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,SparePart,482098,None,1.5,None,2023-09-26T17:09:02.684Z,True
3,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,SparePart,228546,None,1.0,None,2023-09-26T17:09:17.453Z,True
4,04c72e7c-aabb-4055-ac51-0af12068cca5,44441485,SparePart,451630,None,3.1,None,2023-09-26T17:09:25.416Z,True


### CSV Export

In [63]:
# sdf_orders_exploded.toPandas().to_csv('orders.csv', header=True)

## FP Growth

In [48]:
sdf_orders_fp = sdf_orders_exploded.filter(sdf_orders_exploded.TypeId != 'CustomerId').groupBy("OrderNumber").agg(F.collect_set("ProductNumber").alias("SalesNumbers"))

In [133]:
#values_to_check = ['204040', '577474', '576984', '496247', '201464', '200531', '496249', '202314', '203994', '204308']
values_to_check = ['577474', '576984', '201464', '203994', '204308', '203914', '495209', '491498', '205182', '485724']

# Create a filter condition for each value in the list
filter_condition = None
for value in values_to_check:
    if filter_condition is None:
        filter_condition = F.array_contains(sdf_orders_fp.SalesNumbers, value)
    else:
        filter_condition = filter_condition | F.array_contains(sdf_orders_fp.SalesNumbers, value)

# Filter rows where "SalesNumbers" contains any of the specified values
filtered_df = sdf_orders_fp.filter(filter_condition)

In [134]:
filtered_df.toPandas()

,OrderNumber,SalesNumbers
0,16079810,"[10014832, 577039, 491498]"
1,98530340,"[577832, 577753, 201464]"
2,37794386,"[576984, 203992]"
3,89665003,[577474]
4,37779822,"[488543, 577761, 201464, 436520]"
5,54614607,"[577471, 205182, 577759, 577754]"
6,41418926,"[576984, 500119]"
7,13911830,"[495209, 485724]"
8,35863243,"[577315, 577474]"
9,83068967,"[495209, 485724]"


In [146]:
fpGrowth = FPGrowth(itemsCol="SalesNumbers", minSupport=0.05, minConfidence=0.05)
model = fpGrowth.fit(filtered_df)

# Display frequent itemsets.
model.freqItemsets.sort(F.desc("items")).toPandas()

,items,freq
0,"[577821, 201464]",2
1,[577821],2
2,[577474],8
3,"[577315, 577474]",2
4,[577315],2
5,"[577039, 491498]",2
6,[577039],2
7,[576984],6
8,[495209],4
9,[492125],2


In [147]:
# Display generated association rules.
model.associationRules.sort("antecedent", "consequent").toPandas()

,antecedent,consequent,confidence,lift,support
0,[201464],[577821],0.333333,6.666667,0.050
1,[203994],[205398],0.400000,5.333333,0.050
2,[203994],[577474],0.400000,2.000000,0.050
3,[205398],[203994],0.666667,5.333333,0.050
4,[485724],[495209],0.750000,7.500000,0.075
5,[491498],[577039],0.500000,10.000000,0.050
6,[495209],[485724],0.750000,7.500000,0.075
7,[577039],[491498],1.000000,10.000000,0.050
8,[577315],[577474],1.000000,5.000000,0.050
9,[577474],[203994],0.250000,2.000000,0.050


In [148]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(filtered_df).filter(F.size("prediction") > 0).toPandas()

,OrderNumber,SalesNumbers,prediction
0,35863243,"[577315, 577474]",[203994]
1,98530340,"[577832, 577753, 201464]",[577821]
2,84839764,[495209],[485724]
3,37779822,"[488543, 577761, 201464, 436520]",[577821]
4,89665003,[577474],"[203994, 577315]"
5,85228025,"[576984, 577821, 205398, 201464, 577221, 576980]",[203994]
6,37727981,"[490823, 499155, 729418, 201464]",[577821]
7,24279430,"[577315, 577474]",[203994]
8,55629107,"[492125, 201464]",[577821]
9,93386743,"[577366, 577474]","[203994, 577315]"


Pivot the DataFrame again

In [67]:
sdf_orders_pivot = sdf_orders_exploded.filter(sdf_orders_exploded.CustomerId != 'null').groupBy("CustomerId").pivot("ProductNumber").agg(F.count("ProductNumber"))

In [69]:
sdf_orders_pivot.limit(5).toPandas()

,CustomerId,10013518,10029970,10205613,10307709,10699976,10737044,10737172,200243,200531,...,577660,577681,577753,577832,577934,627005,627047,715024,769064,769124
0,21aa4efa-2d73-48ff-8a83-9615e342ab92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3691ab4f-caac-4f53-b708-48e1aa36f89b,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2,fd8ee07e-b5de-4184-bee8-dfc63e2ecaf1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39122c18-7fa8-48a3-92d5-5cc97e8a8c6f,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a7eb0ce4-7f34-4a25-bf60-ac218acc1687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,879cb75b-2568-41aa-b05b-5d3e12d85cf8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,50534e9e-d2fe-4596-a893-1e1b4b020f83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9cd2e1a2-dbd1-42ac-b5fb-f17e259b0792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
9,f9497eaa-7d2d-4fd9-bbbb-3b545d69e31b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


## Join the two sources

In [96]:
sdf_joined = sdf_orders_pivot.join(sdf_assets_pivot, on="CustomerId", how="inner")

In [98]:
spark.conf.set("spark.sql.maxColumns", 300)
import pandas as pd
pd.set_option('display.max_columns', 300)

In [100]:
sdf_joined.limit(3).toPandas()

,CustomerId,10013518,10029970,10205613,10307709,10699976,10737044,10737172,200243,200531,201134,201464,202097,202307,202314,203285,203287,203289,203334,203346,203722,203780,203914,203920,203972,203994,204040,204083,204147,204256,204412,204940,204941,205398,205777,228559,228580,228600,228737,228798,401191,438320,438613,438855,438856,439973,444114,444142,444234,445148,450206,450507,451680,452447,452908,452945,455695,457410,462795,465326,466885,467297,471743,473812,483906,485724,487893,488219,488899,489784,490249,490253,491498,491704,492255,492494,493315,494063,494588,494589,494939,494940,495209,496170,496246,496247,496249,496262,497539,497892,497899,498079,498385,498410,498411,498863,499402,500118,500122,500123,500184,500317,574778,576296,576984,577003,577039,577067,577257,577258,577367,577464,577496,577660,577681,577753,577832,577934,627005,627047,715024,769064,769124,205034,495315,561100,561136,561138,561162,561180,561228,561261,561283,561396,561436,561445,561461,561551,561579,561587,561728,561731,564164,564271,564509,564510,564531,564532,570203,570244,570251,571570,571719,571805,571819,571870,571899,571911,571934,574180,574228,574320,574325,574335,574341,574349,574357,574359,574364,574369,574453,574557,574605,574635,574695,574701,574702,574703,574709,574713,574714,574723,574745,574756,574763,574765,574776,574778,574822,574832,574947,574955,574978,574981,574983,574984,574988,575023,575056,575057,575069,575217,575254,575279,575296,575302,575343,575604,575703,575710,575712,575768,575771,575781,575813,575990,576000,576017,576029,576041,576057,576072,576080,576092,576093,576165,576207,576247,576295,576329,576347,576353,576370,576393,576413,576426,576449,576469,576481,576497,576511,576525,576532,576592,576601,576607,576615,576678,576684,576703,576820,576825,576829,576895,576919,576992,577026,577030,577033,577050,577053,577067,577187,577241,577415,577426,577427,577430,577600,577602,577649,577651,577686,577707,577836,583490,584108,584159,584173,633574,764986,767991,767996,767997,768008,768809,768997,769531,769532,769533
0,f9497eaa-7d2d-4fd9-bbbb-3b545d69e31b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fc8dd9f5-17c9-4788-89c6-29359cc93ae7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [101]:
pd.reset_option('all')
spark.conf.unset("spark.sql.maxColumns")

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/_config/config.py:620: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/_config/config.py:620: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


Spark won't sum the columns directly but create a new columns {name}_1, so we have to sum it afterwards. :|.

There is no such case in the current data.

In [91]:
# TODO ...

#columns_to_sum = [F.col(column_name) for column_name in sdf_orders_pivot.columns if column_name != "CustomerId"]
#for column in columns_to_sum:
#    sdf_joined.withColumn(column.name column + F.col(f"{column.name}_1"))
#    break;
    #sdf_joined = sdf_joined.withColumn(column.name, column + F.col(f"{column.name}_1")).drop(f"{column.name}_1")

AttributeError: 'function' object has no attribute '_get_object_id'